In [1]:
import numpy as np
import time
# Students will submit their files with their team-name.py
# Student have to use the Team as their parent class

Recent Version of the Code

In [115]:
class Agent:
  def __init__(self,num_balls,num_matches,no_wickets=4):
      self.num_balls=num_balls
      self.num_matches=num_matches
      self.actions=np.array([0,1,2,3,4,5])
      self.K=len(self.actions)
      self.no_wickets=no_wickets

      self.arm_counts=np.zeros((self.no_wickets,self.K))
      self.arm_ucb=np.zeros((self.no_wickets,self.K))
      self.arm_rewards= np.zeros((self.no_wickets,self.K))
      self.arm_wickets= np.zeros(self.no_wickets)
      self.current_batter_index= 0
      self.batting_order=np.array([0,1,2,3])

      self.balls=0
      self.innings=0

      self.prev_action=0
      self.action=0

      self.runs_scored_by_batters=np.zeros(self.no_wickets)

      self.strike_rate=np.zeros(self.no_wickets)
      self.average=np.zeros(self.no_wickets)
      self.dismissal_prob=np.zeros(self.no_wickets)

  def check_all_arms_played(self,current_batter_index):
    batter=self.batting_order[current_batter_index]
    for i in range(self.K):
      if self.arm_counts[batter][i]==0:
        return False
    return True



  def get_batting_order(self):
    print(self.balls)
    beta=np.array([0.1,0.1,0.8])

    arr=np.array([self.strike_rate,self.average,self.dismissal_prob])
    arr=arr.T
    arr=arr/(arr.sum(axis=0)+1)
    arr=beta @ arr.T
    out=np.argsort(arr)
    print(out)
    return out

  def get_action(self,wicket,runs_scored):

    if (self.balls==0):
      self.action=np.random.choice(self.actions)
      self.prev_action=self.action
      self.balls+=1
      print(self.action)
      return self.action

    self.arm_rewards[self.batting_order[self.current_batter_index]][self.prev_action]+=runs_scored
    if wicket:
      self.arm_wickets[self.batting_order[self.current_batter_index]]+=1
      self.current_batter_index+=1
      if self.current_batter_index==self.no_wickets:
        self.innings+=1
        self.current_batter_index=0

        self.strike_rate=(np.sum(self.arm_rewards,axis=1)+1)/(np.sum(self.arm_counts,axis=1)+1)
        self.averagee=(np.sum(self.arm_rewards,axis=1)+1)/(self.arm_wickets+1)
        self.dismissal_prob=self.arm_wickets/self.innings

        self.batting_order=self.get_batting_order()
        return 0



    batter=self.batting_order[self.current_batter_index]
    if not self.check_all_arms_played(self.current_batter_index):
      for i in range(self.K):
        if self.arm_counts[self.batting_order[self.current_batter_index]][i]==0:
          self.action=i
          self.prev_action=self.action
          self.arm_counts[self.batting_order[self.current_batter_index]][self.action]+=1
          self.balls+=1
          return self.action

    else:



        self.arm_ucb[batter]= self.arm_rewards[batter]/self.arm_counts[batter] + np.sqrt(4*np.log(self.balls)/self.arm_counts[batter] )

        self.action=np.argmax(self.arm_ucb[batter])
        self.prev_action=self.action
        self.balls+=1
        return self.action




In [116]:
class Environment:
  def __init__(self,num_balls,agent):
    self.num_balls = num_balls
    self.agent = agent
    self.__run_time = 0
    self.__total_runs = 0
    self.__total_wickets = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__p_out =np.random.uniform(0,1,(4,6))
    self.__p_run =np.random.uniform(0,1,(4,6))
    self.__action_runs_map = np.array([0,1,2,3,4,6])
    self.__wickets_left = 4
    self.__wicket = 0
    self.__runs_scored = 0
    self.__start_time = 0
    self.__end_time = 0
    self.__batting_order = np.array([0,1,2,3])
    self.__current_batter = self.__batting_order[self.__wickets_left-1]


  def __get_action(self):
    self.__start_time      = time. time()
    action          = self.agent.get_action(self.__wicket, self.__runs_scored)
    self.__end_time        = time. time()
    self.__run_time   = self.__run_time + self.__end_time - self.__start_time
    return action


  def __get_outcome(self, action):
    self.__current_batter = self.__batting_order[self.__wickets_left-1]
    pout = self.__p_out[self.__current_batter][action]
    prun = self.__p_run[self.__current_batter][action]
    wicket = np.random.choice(2,1,p=[1-pout,pout])[0]
    runs = 0
    if(wicket==0):
      runs = self.__action_runs_map[action]*np.random.choice(2,1,p=[1-prun,prun])[0]
    return wicket, runs


  def innings(self):
    self.__wickets_left = 4
    self.__runs_scored = 0
    self.__batting_order = self.agent.get_batting_order()
    self.__wicket=0

    for ball in range(self.num_balls):
      if (self.__wickets_left > 0 ) :
        action = self.__get_action()
        self.__wicket, self.__runs_scored   = self.__get_outcome(action)
        self.__total_runs     = self.__total_runs + self.__runs_scored
        if (self.__wicket > 0 ):
          self.__wickets_left = self.__wickets_left -1
        self.__total_wickets  = self.__total_wickets + self.__wicket
        if (self.__wickets_left == 0):
          self.__get_outcome(action)
    return self.__total_runs, self.__total_wickets, self.__run_time


In [122]:
num_matches = 10
num_balls = 60
agent = Agent(num_balls,num_matches)
environment = Environment(100,agent)
score = np.zeros((num_matches,1))
run_time = np.zeros((num_matches,1))
wicket = np.zeros((num_matches,1))
for i in range(num_matches):
  score[i],wicket[i],run_time[i] = environment.innings()

  print(score[i])

  print("Innings Over")


0
[0 1 2 3]
4
[4.]
Innings Over
6
[0 1 2 3]
7
[3 1 2 0]
[14.]
Innings Over
13
[3 1 2 0]
17
[3 2 1 0]
[18.]
Innings Over
20
[3 2 1 0]
46
[3 0 1 2]
[48.]
Innings Over
46
[3 0 1 2]
[68.]
Innings Over
57
[3 0 1 2]
61
[3 0 1 2]
[72.]
Innings Over
63
[3 0 1 2]
68
[3 0 1 2]
[76.]
Innings Over
70
[3 0 1 2]
80
[3 0 1 2]
[80.]
Innings Over
82
[3 0 1 2]
[84.]
Innings Over
92
[3 0 1 2]
93
[3 0 1 2]
[84.]
Innings Over


In [58]:
a=np.zeros(4)
b=np.ones(4)
x=np.array([a,b])

In [59]:
x

array([[0., 0., 0., 0.],
       [1., 1., 1., 1.]])

In [110]:
beta=np.array([0.4,0.4,0.2])

arr=np.array([np.ones(4),5*np.ones(4),6*np.ones(4)])

arr=arr.T
arr=arr/arr.sum(axis=0)
arr=beta @ arr.T

In [111]:

out=np.argsort(arr)

out

array([0, 1, 2, 3])